In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
sample_submit = pd.read_csv('./data/sample_submit.csv',header=None, index_col=0)

In [12]:
train_df = train_df.drop(columns=['id'])
df = train_df.dropna()

X = df.drop(columns=['charges'])
y = df['charges']

X = pd.get_dummies(X)

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

# RF

In [108]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_leaf=7, max_features=9)
rf.fit(X_train, y_train)

RandomForestClassifier(max_features=9, min_samples_leaf=7)

In [109]:
from sklearn.metrics import accuracy_score

# 正答率
acc_train = accuracy_score(y_train, rf.predict(X_train))
acc_test = accuracy_score(y_test, rf.predict(X_test))

print("acc_train: "+ str(acc_train) + "   acc_test: "+ str(acc_test))

acc_train: 0.90703125   acc_test: 0.871875


In [110]:
from sklearn.metrics import f1_score

f1_train = f1_score(y_train, rf.predict(X_train), average='macro')
f1_test = f1_score(y_test, rf.predict(X_test), average='macro')

# 評価結果の表示
print("f1_train:", f1_train, "f1_test", f1_test)

f1_train: 0.8038054775203202 f1_test 0.7047950564798956


In [104]:
from sklearn.metrics import confusion_matrix

cm_train = confusion_matrix(y_train, rf.predict(X_train))
cm_test = confusion_matrix(y_test, rf.predict(X_test))

# 混同行列の表示
print(cm_train)
print(cm_test)

[[968  28   9]
 [ 63  80  15]
 [  8   9 100]]
[[241   8   2]
 [ 17  15   8]
 [  1   9  19]]


In [88]:
for min_samples_leaf in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
            rfc = RandomForestClassifier(n_estimators=500, min_samples_leaf=min_samples_leaf, random_state=0)
            rfc.fit(X_train, y_train)
            score = rfc.score(X_test, y_test)
            f1 = f1_score(y_test, rfc.predict(X_test), average='macro')
            print(("min_samples_leaf={} ""score={:.4f}""f1={:.4f}").format(min_samples_leaf, score, f1))

min_samples_leaf=1 score=0.8375f1=0.6326
min_samples_leaf=2 score=0.8500f1=0.6487
min_samples_leaf=3 score=0.8594f1=0.6700
min_samples_leaf=4 score=0.8656f1=0.6861
min_samples_leaf=5 score=0.8656f1=0.6809
min_samples_leaf=6 score=0.8688f1=0.6889
min_samples_leaf=7 score=0.8594f1=0.6723
min_samples_leaf=8 score=0.8594f1=0.6792
min_samples_leaf=9 score=0.8562f1=0.6697
min_samples_leaf=10 score=0.8562f1=0.6697


In [96]:
from sklearn.model_selection import GridSearchCV
param_range = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
param_grid = {"max_features": param_range, "min_samples_leaf": param_range}

rfc = RandomForestClassifier(n_estimators=500, random_state=0, max_depth=None)
gs_rf = GridSearchCV(estimator=rfc, param_grid=param_grid, scoring='accuracy', cv=5)

gs_rf.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(n_estimators=500, random_state=0),
             param_grid={'max_features': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
             scoring='accuracy')

In [98]:
gs_rf.best_score_, gs_rf.best_params_

(0.87578125, {'max_features': 4, 'min_samples_leaf': 9})

In [100]:
clf = gs_rf.best_estimator_
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
f1_score(y_test, y_pred, average='macro')

0.6833736234219715

## 提出

In [111]:
test = test_df.drop(columns=['id'])
#print(test.shape)

#test = test.fillna(0)
#print(test.shape)

test_X = pd.get_dummies(test)
test_X.shape

(400, 11)

In [ ]:
sample_submit[1] = pipe.predict(test_X)
sample_submit.to_csv('submit.csv', header=None)
sample_submit.head()